In [7]:
# Advanced Pandas and NumPy with NLP Analysis and Clustering

## 1. Setup and Imports

import numpy as np  # NumPy for numerical operations and array manipulation
import pandas as pd  # Pandas for data manipulation and analysis
import matplotlib.pyplot as plt  # For visualization
import seaborn as sns  # Enhanced visualizations
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.metrics import silhouette_score
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import re
import string
from collections import Counter

import nltk
from nltk.tokenize import word_tokenize

# Download required NLTK data resources with explicit verification
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

# Verify the punkt tokenizer is available
try:
    from nltk.data import find
    find('tokenizers/punkt')
    print("Punkt tokenizer found successfully!")
except LookupError:
    print("Punkt tokenizer not found. Downloading again...")
    nltk.download('punkt', quiet=False)

print("Libraries imported successfully!")


Punkt tokenizer found successfully!
Libraries imported successfully!


[nltk_data] Downloading package punkt to
[nltk_data]     /Users/salkimmich/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/salkimmich/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/salkimmich/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:

## 2. Creating Sample Survey Data

# Let's create a synthetic dataset similar to your Listen4Good NLP analysis
np.random.seed(42)  # NumPy's random seed ensures reproducibility

# Create sample survey responses
responses = [
    "The service at this clinic was excellent. Staff was very responsive and caring.",
    "Wait times were too long. I had to sit for over an hour before seeing a doctor.",
    "Doctors were knowledgeable but the facility needs updating.",
    "I appreciate how the staff explained everything clearly to me.",
    "The parking situation is terrible. Had to walk far with my injured leg.",
    "Very clean facility and professional staff. Would recommend to others.",
    "Communication could be improved. I wasn't notified about my appointment change.",
    "The children's play area was a nice touch while waiting for our appointment.",
    "Billing department made multiple errors and was difficult to reach.",
    "The new patient portal is confusing and hard to navigate.",
    "Nurses were attentive and made me feel comfortable during my procedure.",
    "I was surprised by the unexpected costs that weren't explained beforehand.",
    "The follow-up care instructions were clear and helpful.",
    "Front desk staff needs more training on insurance procedures.",
    "I felt rushed during my appointment. Doctor barely spent 5 minutes with me.",
    "The telehealth option saved me so much time. Very convenient.",
    "Equipment seemed outdated compared to other facilities I've visited.",
    "Staff was friendly but seemed understaffed and overworked.",
    "The specialty care I received for my condition was exceptional.",
    "Difficult to get appointments in a reasonable timeframe.",
    "The clinic location is convenient but hours are too limited.",
    "My treatment plan was thoroughly explained and I felt involved in decisions.",
    "Waiting room was overcrowded with sick patients too close together.",
    "Reception staff was rude when I asked questions about my insurance.",
    "The pharmacy coordination with my doctor worked seamlessly.",
    "More diverse language options would help many patients in this community.",
    "Billing was straightforward and transparent. No surprises.",
    "The pediatric nurse was amazing with my anxious child.",
    "Too many forms to fill out that asked for redundant information.",
    "The online scheduling system repeatedly crashed when I tried to use it."
]

# Create demographic and metadata
# IMPORTANT: Using NumPy's random functions for generating synthetic data
# is much faster than Python's random module for large datasets
demographics = []
for i in range(len(responses)):
    # np.random.choice is faster than random.choice for selecting from arrays
    age_group = np.random.choice(['18-24', '25-34', '35-44', '45-54', '55-64', '65+'])
    gender = np.random.choice(['Female', 'Male', 'Other', 'Prefer not to say'])
    visit_type = np.random.choice(['Primary Care', 'Specialist', 'Emergency', 'Routine Checkup', 'Procedure'])
    satisfaction = np.random.randint(1, 11)  # 1-10 satisfaction score
    
    demographics.append({
        'response_id': i+1,
        'age_group': age_group,
        'gender': gender,
        'visit_type': visit_type,
        'satisfaction_score': satisfaction
    })

# Create DataFrames - Pandas makes it easy to organize structured data
# WHY PANDAS: We use DataFrames here because they handle mixed data types
# (text, categories, numbers) in a tabular structure with labeled columns
df_responses = pd.DataFrame({
    'response_id': range(1, len(responses)+1),
    'feedback_text': responses
})

df_demographics = pd.DataFrame(demographics)

print("Sample data created!")
print(f"Number of responses: {len(df_responses)}")
df_responses.head(3)  # Pandas .head() method is great for quickly viewing data samples

Sample data created!
Number of responses: 30


,response_id,feedback_text
0,1,The service at this clinic was excellent. Staf...
1,2,Wait times were too long. I had to sit for ove...
2,3,Doctors were knowledgeable but the facility ne...


In [3]:
## 3. NumPy Fundamentals with Data Preparation

# WHY NUMPY: Convert Pandas Series to NumPy array for faster numerical operations
# NumPy arrays are more memory-efficient and perform calculations faster
satisfaction_scores = np.array(df_demographics['satisfaction_score'])

print("\nSatisfaction Scores (first 10):", satisfaction_scores[:10])
print("Shape:", satisfaction_scores.shape)  # NumPy arrays have convenient shape properties
print("Data Type:", satisfaction_scores.dtype)  # NumPy manages data types efficiently

# Basic NumPy statistics on our data
# WHY NUMPY: These statistical functions are vectorized (no loops needed)
# and much faster than calculating these manually
print("\nBasic Statistics with NumPy:")
print("Mean satisfaction:", np.mean(satisfaction_scores))
print("Median satisfaction:", np.median(satisfaction_scores))
print("Standard deviation:", np.std(satisfaction_scores))
print("Min score:", np.min(satisfaction_scores))
print("Max score:", np.max(satisfaction_scores))

# Create a frequency distribution using NumPy
# IMPORTANT: np.unique efficiently finds unique values and counts them
# This is much faster than using Python loops or collections.Counter
unique_scores, counts = np.unique(satisfaction_scores, return_counts=True)
print("\nFrequency Distribution:")
for score, count in zip(unique_scores, counts):
    print(f"Score {score}: {count} responses")

# Converting to percentiles with NumPy
# WHY NUMPY: np.percentile calculates multiple percentiles in one pass through the data
percentiles = np.percentile(satisfaction_scores, [25, 50, 75, 90])
print("\nPercentiles:")
print(f"25th percentile: {percentiles[0]}")
print(f"50th percentile (median): {percentiles[1]}")
print(f"75th percentile: {percentiles[2]}")
print(f"90th percentile: {percentiles[3]}")

# Using NumPy for filtering and conditional operations
# WHY NUMPY: Boolean indexing in NumPy is very efficient and readable
# Compare to having to write a loop or comprehension in pure Python
high_scores = satisfaction_scores[satisfaction_scores >= 8]
low_scores = satisfaction_scores[satisfaction_scores <= 3]

print(f"\nNumber of high scores (8-10): {len(high_scores)}")
print(f"Number of low scores (1-3): {len(low_scores)}")
print(f"Percentage of high scores: {len(high_scores)/len(satisfaction_scores)*100:.1f}%")


Satisfaction Scores (first 10): [8 3 4 2 5 6 1 3 9 7]
Shape: (30,)
Data Type: int64

Basic Statistics with NumPy:
Mean satisfaction: 5.166666666666667
Median satisfaction: 4.0
Standard deviation: 3.066847820736392
Min score: 1
Max score: 10

Frequency Distribution:
Score 1: 3 responses
Score 2: 5 responses
Score 3: 4 responses
Score 4: 4 responses
Score 5: 1 responses
Score 6: 1 responses
Score 7: 3 responses
Score 8: 2 responses
Score 9: 4 responses
Score 10: 3 responses

Percentiles:
25th percentile: 2.25
50th percentile (median): 4.0
75th percentile: 8.0
90th percentile: 9.100000000000001

Number of high scores (8-10): 9
Number of low scores (1-3): 12
Percentage of high scores: 30.0%


In [4]:
## 4. Pandas Data Manipulation and Analysis

# Merging our response and demographic DataFrames
# WHY PANDAS: pd.merge provides SQL-like join operations that would be complex to implement manually
df_merged = pd.merge(df_responses, df_demographics, on='response_id')
print("\nMerged DataFrame (first 3 rows):")
print(df_merged.head(3))

# Basic exploratory data analysis with Pandas
# IMPORTANT: Pandas provides simple methods for data exploration
print("\nDataset Overview:")
print(df_merged.info())  # Shows data types and missing values count

print("\nSummary Statistics:")
# WHY PANDAS: describe() calculates multiple statistics at once,
# handling different data types appropriately
print(df_merged.describe(include='all'))

# Group analysis with Pandas
# WHY PANDAS: groupby() is a powerful operation that splits data into groups
# and performs aggregate operations in one step
print("\nSatisfaction by Visit Type:")
visit_satisfaction = df_merged.groupby('visit_type')['satisfaction_score'].agg(['mean', 'median', 'count', 'std'])
print(visit_satisfaction.sort_values('mean', ascending=False))  # Sorting with Pandas is straightforward

print("\nSatisfaction by Age Group:")
age_satisfaction = df_merged.groupby('age_group')['satisfaction_score'].agg(['mean', 'median', 'count', 'std'])
print(age_satisfaction.sort_values('mean', ascending=False))

print("\nSatisfaction by Gender:")
gender_satisfaction = df_merged.groupby('gender')['satisfaction_score'].agg(['mean', 'median', 'count', 'std'])
print(gender_satisfaction)

# Create satisfaction categories for analysis
# WHY PANDAS: apply() lets us run functions on every row or column efficiently
def categorize_satisfaction(score):
    if score >= 8:
        return 'High (8-10)'
    elif score >= 5:
        return 'Medium (5-7)'
    else:
        return 'Low (1-4)'

df_merged['satisfaction_category'] = df_merged['satisfaction_score'].apply(categorize_satisfaction)

# Cross-tabulation with Pandas
# WHY PANDAS: crosstab provides an efficient way to count frequencies across two variables
print("\nCross-tabulation of Visit Type and Satisfaction Category:")
visit_sat_crosstab = pd.crosstab(
    df_merged['visit_type'], 
    df_merged['satisfaction_category'],
    normalize='index'  # Normalize to get percentages by row
) * 100  # Convert to percentages

print(visit_sat_crosstab.round(1))  # Pandas makes rounding easy



Merged DataFrame (first 3 rows):
   response_id                                      feedback_text age_group  \
0            1  The service at this clinic was excellent. Staf...     45-54   
1            2  Wait times were too long. I had to sit for ove...     55-64   
2            3  Doctors were knowledgeable but the facility ne...     35-44   

   gender  visit_type  satisfaction_score  
0  Female   Emergency                   8  
1  Female  Specialist                   3  
2   Other   Procedure                   4  

Dataset Overview:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30 entries, 0 to 29
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   response_id         30 non-null     int64 
 1   feedback_text       30 non-null     object
 2   age_group           30 non-null     object
 3   gender              30 non-null     object
 4   visit_type          30 non-null     object
 5   satisfactio

In [5]:
## 5. Text Preprocessing for NLP Analysis

# Define text preprocessing function
def preprocess_text(text):
    # Convert to lowercase
    text = text.lower()
    
    # Remove punctuation
    text = re.sub(f'[{string.punctuation}]', ' ', text)
    
    # Remove numbers
    text = re.sub(r'\d+', '', text)
    
    # Tokenize
    tokens = word_tokenize(text)
    
    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    tokens = [token for token in tokens if token not in stop_words and len(token) > 2]
    
    # Lemmatize
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(token) for token in tokens]
    
    return ' '.join(tokens)

# Apply preprocessing to our feedback text
# WHY PANDAS: apply() method allows applying a function to each value in a Series
# This avoids writing explicit loops and makes the code more readable
df_merged['processed_text'] = df_merged['feedback_text'].apply(preprocess_text)

print("\nOriginal vs Processed Text (first 3 examples):")
for i in range(3):
    print(f"\nOriginal: {df_merged['feedback_text'].iloc[i]}")
    print(f"Processed: {df_merged['processed_text'].iloc[i]}")


LookupError: 
**********************************************************************
  Resource [93mpunkt_tab[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('punkt_tab')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtokenizers/punkt_tab/english/[0m

  Searched in:
    - '/Users/salkimmich/nltk_data'
    - '/opt/anaconda3/nltk_data'
    - '/opt/anaconda3/share/nltk_data'
    - '/opt/anaconda3/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
**********************************************************************


In [ ]:
## 6. Feature Extraction with TF-IDF

# Create TF-IDF features
tfidf_vectorizer = TfidfVectorizer(
    max_features=100,  # Limit to top 100 features for this example
    min_df=2           # Ignore terms that appear in less than 2 documents
)

# Fit and transform the processed text
# IMPORTANT: The output is a sparse matrix (not a standard NumPy array)
# This is more memory efficient for text data with many zeros
tfidf_matrix = tfidf_vectorizer.fit_transform(df_merged['processed_text'])

# Convert to a pandas DataFrame for easier inspection
# WHY PANDAS: Converting to DataFrame adds column names and makes the data more interpretable
tfidf_df = pd.DataFrame(
    tfidf_matrix.toarray(),  # Convert sparse matrix to dense NumPy array
    columns=tfidf_vectorizer.get_feature_names_out()
)

print("\nTF-IDF Matrix Shape:", tfidf_matrix.shape)
print("Sample of TF-IDF Features (first 5 columns, first 3 rows):")
print(tfidf_df.iloc[:3, :5])  # Pandas indexing makes it easy to view subsets of data

# Get the most important terms by their TF-IDF scores
# WHY NUMPY: np.sum along axis=0 efficiently sums columns
feature_importance = np.sum(tfidf_matrix.toarray(), axis=0)
feature_names = tfidf_vectorizer.get_feature_names_out()

# Create a DataFrame of terms and their importance
# WHY PANDAS: DataFrames make it easy to organize and sort results
term_importance_df = pd.DataFrame({
    'term': feature_names,
    'importance': feature_importance
})

print("\nTop 10 Most Important Terms:")
print(term_importance_df.sort_values('importance', ascending=False).head(10))

In [ ]:

## 7. Clustering Analysis with K-means

# Determine optimal number of clusters using silhouette scores
silhouette_scores = []
k_range = range(2, 11)  # Try 2 to 10 clusters

for k in k_range:
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
    cluster_labels = kmeans.fit_predict(tfidf_matrix)
    
    # WHY NUMPY: Silhouette score calculation uses efficient array operations
    silhouette_avg = silhouette_score(tfidf_matrix, cluster_labels)
    silhouette_scores.append(silhouette_avg)
    print(f"For n_clusters = {k}, the silhouette score is {silhouette_avg:.3f}")

# Find the optimal k
# WHY NUMPY: np.argmax finds the index of the maximum value in an array efficiently
optimal_k = k_range[np.argmax(silhouette_scores)]
print(f"\nOptimal number of clusters based on silhouette score: {optimal_k}")

# Apply KMeans with the optimal number of clusters
kmeans = KMeans(n_clusters=optimal_k, random_state=42, n_init=10)
# WHY PANDAS: We can easily add the cluster assignments as a new column in our DataFrame
df_merged['cluster'] = kmeans.fit_predict(tfidf_matrix)

# Analyze clusters
# WHY PANDAS: value_counts() efficiently counts occurrences of each unique value
cluster_counts = df_merged['cluster'].value_counts().sort_index()
print("\nNumber of responses in each cluster:")
print(cluster_counts)

# Calculate mean satisfaction by cluster
# WHY PANDAS: groupby() lets us easily calculate statistics for each cluster
cluster_satisfaction = df_merged.groupby('cluster')['satisfaction_score'].mean().sort_index()
print("\nMean satisfaction score by cluster:")
print(cluster_satisfaction)


In [ ]:
## 8. Visualizing the Clusters with PCA

# Reduce dimensionality for visualization
# WHY NUMPY: PCA works directly with NumPy arrays and uses efficient linear algebra operations
pca = PCA(n_components=2)
pca_result = pca.fit_transform(tfidf_matrix.toarray())

# Create a DataFrame for plotting
# WHY PANDAS: Organizing our visualization data in a DataFrame makes plotting easier
plot_df = pd.DataFrame({
    'x': pca_result[:, 0],  # First principal component
    'y': pca_result[:, 1],  # Second principal component
    'cluster': df_merged['cluster'],
    'satisfaction': df_merged['satisfaction_score']
})

print("\nPCA Results (first 5 rows):")
print(plot_df.head())

# Plotting code (uncomment to use in Jupyter)
# plt.figure(figsize=(12, 8))
# sns.scatterplot(data=plot_df, x='x', y='y', hue='cluster', palette='viridis', 
#                size='satisfaction', sizes=(20, 200), alpha=0.7)
# plt.title('PCA Visualization of Response Clusters')
# plt.xlabel('Principal Component 1')
# plt.ylabel('Principal Component 2')
# plt.legend(title='Cluster', bbox_to_anchor=(1.05, 1), loc='upper left')
# plt.tight_layout()
# plt.show()

In [ ]:
## 9. Understanding Cluster Content

# Function to get top terms for each cluster
def get_cluster_top_terms(kmeans_model, vectorizer, cluster_id, top_n=10):
    # Get cluster center
    # WHY NUMPY: Cluster centers are stored as NumPy arrays for efficient computation
    center = kmeans_model.cluster_centers_[cluster_id]
    
    # Get feature names
    feature_names = vectorizer.get_feature_names_out()
    
    # Get indices of top n values in the cluster center
    # WHY NUMPY: argsort() efficiently returns indices that would sort the array
    # Using negative indices and reversing gets us the highest values
    top_indices = center.argsort()[-top_n:][::-1]
    
    # Get the terms and their weights
    top_terms = [(feature_names[i], center[i]) for i in top_indices]
    
    return top_terms

# Get top terms for each cluster
print("\nTop Terms by Cluster:")
for i in range(optimal_k):
    top_terms = get_cluster_top_terms(kmeans, tfidf_vectorizer, i)
    print(f"\nCluster {i}:")
    for term, weight in top_terms:
        print(f"  {term}: {weight:.3f}")

# Extract sample responses from each cluster
# WHY PANDAS: Boolean indexing makes it easy to filter rows by cluster
print("\nSample Responses from Each Cluster:")
for i in range(optimal_k):
    cluster_resp = df_merged[df_merged['cluster'] == i]
    print(f"\nCluster {i} (n={len(cluster_resp)}, avg satisfaction={cluster_resp['satisfaction_score'].mean():.2f}):")
    
    # Show 2 example responses from this cluster
    # WHY PANDAS: sample() method easily selects random rows
    examples = cluster_resp.sample(min(2, len(cluster_resp)))
    for j, row in examples.iterrows():
        print(f"  - \"{row['feedback_text']}\" (satisfaction: {row['satisfaction_score']})")

In [ ]:
## 10. Putting It All Together: Insights Dashboard

# Create a summary of our findings for a dashboard
# WHY PANDAS: Methods like mean(), value_counts(), and to_dict() make summarizing data easy
summary = {
    'total_responses': len(df_merged),
    'avg_satisfaction': df_merged['satisfaction_score'].mean(),
    'satisfaction_distribution': df_merged['satisfaction_category'].value_counts().to_dict(),
    'visit_types': df_merged['visit_type'].value_counts().to_dict(),
    'num_clusters': optimal_k,
    'cluster_sizes': cluster_counts.to_dict(),
    'cluster_satisfaction': cluster_satisfaction.to_dict()
}

print("\n=== INSIGHTS DASHBOARD ===")
print(f"Total Responses Analyzed: {summary['total_responses']}")
print(f"Average Satisfaction Score: {summary['avg_satisfaction']:.2f}/10")

print("\nSatisfaction Distribution:")
for category, count in summary['satisfaction_distribution'].items():
    percentage = count / summary['total_responses'] * 100
    print(f"  {category}: {count} responses ({percentage:.1f}%)")

print("\nVisit Type Distribution:")
for visit_type, count in summary['visit_types'].items():
    percentage = count / summary['total_responses'] * 100
    print(f"  {visit_type}: {count} responses ({percentage:.1f}%)")

print(f"\nIdentified {summary['num_clusters']} distinct feedback themes:")
for cluster, count in summary['cluster_sizes'].items():
    sat = summary['cluster_satisfaction'][cluster]
    percentage = count / summary['total_responses'] * 100
    print(f"  Cluster {cluster}: {count} responses ({percentage:.1f}%), Avg Satisfaction: {sat:.2f}/10")

print("\nRecommended Actions:")
for cluster in range(optimal_k):
    cluster_sat = summary['cluster_satisfaction'][cluster]
    if cluster_sat < 5:
        print(f"  - Address issues in Cluster {cluster} (low satisfaction area)")
    elif cluster_sat > 8:
        print(f"  - Leverage strengths from Cluster {cluster} (high satisfaction area)")

print("\nAnalysis Complete!")